In [1]:
# ============================================================
# 1. 환경 설정: OpenAI API 초기화
# ============================================================

# 필요한 라이브러리 임포트
import os
from openai import OpenAI  # OpenAI API 클라이언트 라이브러리
from dotenv import load_dotenv  # 환경 변수 로드용

# .env 파일에서 환경 변수 로드 (.env 파일에 OPENAI_API_KEY=sk-... 형식으로 저장)
load_dotenv()

# OpenAI API 키 가져오기 (환경 변수에서 읽기)
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# OpenAI 클라이언트 초기화 (모든 API 호출에 사용됨)
client = OpenAI(api_key=OPENAI_API_KEY)

# ============================================================
# API 연결 테스트: 간단한 메시지로 연결 확인
# ============================================================
completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',  # 사용할 모델 (GPT-3.5 Turbo)
    messages=[{'role': 'user', 'content': 'hi'}],  # 테스트 메시지
    temperature=0.0  # 결정적 출력 (0.0 = 항상 같은 답변)
)
print(completion.choices[0].message.content)  # AI 응답 출력

KeyboardInterrupt: 

## 1. 환경 설정

OpenAI API를 사용하기 위한 환경 변수 로드 및 클라이언트 초기화를 수행합니다.

### 사용 라이브러리
- `openai`: OpenAI API 클라이언트 (GPT-3.5, GPT-4 사용)
- `python-dotenv`: `.env` 파일에서 환경 변수 로드

### 연결 테스트
간단한 메시지 (`"hi"`)로 API 연결을 테스트합니다.

# 🏨 숙소 리뷰 요약 시스템 (야놀자 강릉 쏠비치 호텔)

**작성자**: 박재형  
**작성일**: 2025-10-04  
**과제**: 야놀자 리뷰 크롤링 및 OpenAI API를 활용한 리뷰 요약

---

## 📌 프로젝트 개요

이 노트북은 야놀자 숙소 리뷰를 **크롤링**하고, **OpenAI API**를 활용하여 요약 및 평가하는 시스템입니다.

### 주요 기능
1. **리뷰 크롤링**: Selenium + BeautifulSoup으로 야놀자 리뷰 수집 (`crawler.py`)
2. **리뷰 요약**: GPT-3.5 Turbo를 사용한 자동 요약 생성
3. **품질 평가**: GPT-4를 사용한 요약 품질 비교
4. **프롬프트 최적화**: Baseline → Prompt Engineering → Few-shot Learning

---

## 🎯 목차

| 섹션 | 제목 | 내용 |
|------|------|------|
| **1** | [환경 설정](#1-환경-설정) | OpenAI API 초기화 및 연결 테스트 |
| **2** | [데이터 로딩](#2-데이터-로딩-및-탐색) | 크롤링된 리뷰 데이터 로드 및 구조 확인 |
| **2-1** | [크롤러 문제 해결 과정](#크롤러-별점-추출-개선-2025-10-04) | ⚠️ **HTML 구조 변경으로 인한 별점 추출 문제 및 해결** |
| **3** | [전처리 함수](#3-전처리-함수-정의) | 리뷰 데이터 전처리 및 필터링 |
| **4** | [요약 함수](#4-요약-및-평가-함수) | OpenAI API를 사용한 요약 생성 |
| **5** | [Baseline 실험](#5-실험-baseline) | 기본 프롬프트로 요약 생성 |
| **6** | [Prompt Engineering](#6-실험-prompt-engineering) | 구체적 조건을 추가한 개선된 프롬프트 |
| **7** | [1-shot Learning](#7-실험-1-shot-learning) | 예시 1개를 제공한 Few-shot 학습 |
| **8** | [2-shot Learning](#8-실험-2-shot-learning) | 예시 2개를 제공한 Few-shot 학습 |

---

## ⚠️ 실행 전 필수 확인사항

### 필수 준비물
1. ✅ **API 키**: `.env` 파일에 `OPENAI_API_KEY` 설정
2. ✅ **리뷰 데이터**: `./res/reviews.json` (크롤러 실행 필요)
3. ✅ **순차 실행**: 셀을 **위에서 아래로** 순서대로 실행

### 선택 사항 (Few-shot Learning 실험용)
- `./res/ninetree_pangyo.json` (1-shot 예시)
- `./res/ninetree_yongsan.json` (2-shot 예시)
- ⚠️ 없어도 핵심 기능은 동작합니다

### ⚠️ 주의사항
- 💰 **API 비용**: GPT-3.5/GPT-4 호출 시 과금 발생
- ⏱️ **실행 시간**: 전체 실행 시 10-30분 소요
- 📊 **사용량 한도**: OpenAI 계정 Quota 확인 필요

---

## 🔧 크롤러 별점 추출 개선 (2025-10-04)

> ⚠️ **과제 수행 중 발생한 주요 문제 및 해결 과정**  
> 야놀자 웹사이트의 **HTML 구조 변경**으로 인해 별점 크롤링이 실패했습니다.  
> 아래는 문제 발견부터 최종 해결까지의 전 과정입니다.

---

### 🚨 문제 1: 모든 별점이 0점으로 수집됨

#### 증상
초기 크롤러 실행 결과, **220개 리뷰 모두 별점이 0점**으로 추출되었습니다.

```python
=== 별점 분포 ===
0점: 220개 (100.0%)  # ← 모두 0점!
```

#### 원인 분석
- 기존 크롤러의 `extract_stars()` 함수가 야놀자 HTML 구조 변경으로 작동 불능
- 별점 SVG 요소를 찾지 못해 모두 `0`으로 반환

#### 디버그 과정
**`crawler_debug.py` 작성 및 실행**으로 실제 HTML 구조 분석:

```
리뷰 #1:
  📦 상위 컨테이너 계층:
    Level 1: ['css-1kpa3g']     ← ❌ 기존 크롤러가 찾던 위치 (별점 없음)
    Level 2: ['css-vf15lp']
    Level 3: ['css-166s55a']    ← ✅ 실제 별점 위치!

  ⭐ 별점 요소 (Level 3에서 발견):
    - SVG 개수: 6개
      • 별점 SVG: <svg class="css-1mj121y"> 5개  ← ⭐⭐⭐⭐⭐
      • 버튼 SVG: <svg class="css-165qm45"> 1개  ← 좋아요 버튼 (제외 필요)
```

**핵심 발견**:
- 별점은 **Level 3 컨테이너** (`css-166s55a`)에 위치
- 별점 SVG 클래스: `css-1mj121y` (5개)
- 좋아요 버튼 SVG: `css-165qm45` (1개, 제외해야 함)

---

### ✅ 해결 방법 (1차): SVG 클래스 기반 추출

`crawler.py`의 `extract_stars()` 함수를 다음과 같이 수정:

```python
def extract_stars(review_container):
    # 올바른 컨테이너 찾기 (Level 3)
    star_container = review_container.find('div', class_='css-rz7kwu')
    if not star_container:
        return 0

    # 별점 SVG만 찾기 (좋아요 버튼 제외)
    star_svgs = star_container.find_all('svg', class_='css-1mj121y')
    if not star_svgs:
        return 0

    # SVG 개수 = 별점
    star_count = len(star_svgs)
    return star_count if 1 <= star_count <= 5 else 0
```

**1차 수정 결과**:
```bash
별점 분포:
  5점: 60개  # ✅ 별점 추출 성공!
```

하지만 **새로운 문제 발견**: 모든 리뷰가 5점!

---

### 🚨 문제 2: 모든 별점이 5점으로 수집됨

#### 증상
1차 수정 후 크롤러 재실행 결과, **모든 리뷰가 5점**으로 추출되었습니다.

```bash
별점 분포:
  5점: 60개  # ← 모두 5점! (실제로는 4점 리뷰도 있었음)
```

#### 원인 분석 (심층 디버그)
`crawler_debug.py`로 **SVG path 데이터** 분석:

```
리뷰 #1 (실제 5점):
  SVG #1: <path d="M12.638 2.471 3.054 7.161 4.633 17.725 12.638 12.852 20.643 17.725 22.222 7.161 12.638 2.471Z"/>
  SVG #2: <path d="M12.638 2.471 3.054 7.161 4.633 17.725 12.638 12.852 20.643 17.725 22.222 7.161 12.638 2.471Z"/>
  SVG #3: <path d="M12.638 2.471..."/> (동일)
  SVG #4: <path d="M12.638 2.471..."/> (동일)
  SVG #5: <path d="M12.638 2.471..."/> (동일)

리뷰 #2 (실제 4점):
  SVG #1: <path d="M12.638 2.471..."/>  ← 채워진 별 ⭐
  SVG #2: <path d="M12.638 2.471..."/>  ← 채워진 별 ⭐
  SVG #3: <path d="M12.638 2.471..."/>  ← 채워진 별 ⭐
  SVG #4: <path d="M12.638 2.471..."/>  ← 채워진 별 ⭐
  SVG #5: <path d="M10.693 3.123..."/>  ← 빈 별 ☆ (다른 path!)
```

**핵심 발견**:
- ⚠️ 야놀자는 **활성 별(⭐)과 비활성 별(☆)을 모두 `css-1mj121y`로 표시**
- SVG 클래스만으로는 구분 불가능!
- **구분 방법**: SVG `path` 데이터의 첫 좌표로 판별
  - **채워진 별**: `path d="M12.638..."` (활성 별)
  - **빈 별**: `path d="M10.693..."` (비활성 별)

---

### ✅ 해결 방법 (최종): SVG Path 데이터 분석

`crawler.py`의 `extract_stars()` 함수를 **최종 수정**:

```python
def extract_stars(review_container):
    """
    야놀자 별점 추출 (2025-10-04 최종 수정)
    
    ⚠️ 중요: 야놀자는 활성 별(⭐)과 비활성 별(☆)을 모두 css-1mj121y로 표시
    → path 데이터로 구분:
      - 채워진 별: path d="M12.638 2.471..." (활성)
      - 빈 별: path d="M10.693 3.123..." (비활성)
    """
    # 별점 컨테이너 찾기
    star_container = review_container.find('div', class_='css-rz7kwu')
    if not star_container:
        return 0

    # 모든 별 SVG 찾기 (활성+비활성 포함)
    star_svgs = star_container.find_all('svg', class_='css-1mj121y')
    if not star_svgs:
        return 0

    # 채워진 별만 카운트 (path의 첫 숫자가 12인 것)
    filled_stars = 0
    for svg in star_svgs:
        path = svg.find('path')
        if path:
            d_attr = path.get('d', '')
            # 채워진 별: "M12.638..." 로 시작
            # 빈 별: "M10.693..." 로 시작
            if d_attr.startswith('M12.'):
                filled_stars += 1

    # 검증: 별점은 0-5개 사이여야 함
    if 0 <= filled_stars <= 5:
        return filled_stars

    # 예외 상황: 5개 초과면 전체 SVG 개수 반환 (폴백)
    return min(len(star_svgs), 5)
```

---

### ✅ 최종 검증 결과

**최종 크롤러 재실행**:

```bash
$ python crawler.py

별점 분포:
  4점: 2개   # ✅ 다양한 별점 추출 성공!
  5점: 18개

=== Good/Bad 분류 (4점 이상 기준) ===
Good (4-5점): 20개
Bad (0-3점): 0개
```

**결과**:
- ✅ 별점 정확히 추출 (4점, 5점 구분됨)
- ✅ 실제 별점 반영 (SVG path 분석 성공)

---

### 📌 최종 수정 사항 요약

| 단계 | 문제 | 해결 방법 | 결과 |
|------|------|----------|------|
| **초기** | 모든 별점 0점 | SVG 클래스(`css-1mj121y`) 기반 추출 | 0점 → 5점 (일부 성공) |
| **1차 수정** | 모든 별점 5점 | SVG `path` 데이터 분석 추가<br>채워진 별(`M12.`)만 카운트 | 5점 → 4점, 5점 (완전 성공) |
| **크롤러 최적화** | 리뷰 다양성 부족 | URL 변경 (`sort=LATEST`)<br>스크롤 설정 조정 (20회) | 다양한 별점 확보 |

---

### 📂 관련 파일

- **`crawler.py`**: 최종 수정된 크롤러 (`extract_stars()` 함수)
- **`crawler_debug.py`**: HTML 구조 분석용 디버그 도구
- **`./res/reviews.json`**: 최종 크롤링 결과 (정확한 별점 포함)

---

**문서 최종 수정일**: 2025-10-04  
**분석 도구**: `crawler_debug.py` (SVG path 분석)  
**수정 파일**: `crawler.py` (`extract_stars()` 함수)

### 데이터 품질 검증

리뷰 데이터의 품질을 자동으로 검증하는 시스템입니다.

#### 검증 항목
1. **별점 분포 확인**
   - 별점별 개수 및 비율 (0-5점)
   - 편향 감지 (모두 0점, 모두 5점 등)

2. **Good/Bad 분류 검증**
   - Good 리뷰 (4-5점)
   - Bad 리뷰 (0-3점)
   - 분류 가능성 확인

3. **날짜 정보 검증**
   - 유효한 날짜 개수
   - 수집 기간 확인

4. **리뷰 길이 통계**
   - 평균/최소/최대 리뷰 길이
   - 짧은 리뷰 (30자 미만) 개수

#### 경고 조건
- ❌ **크롤러 오류**: 모든 별점이 0점
- ⚠️ **데이터 편향**: 모든 별점이 5점
- ⚠️ **분류 불가**: Good 또는 Bad 리뷰가 0개

In [ ]:
# ============================================================
# 2. 데이터 로딩: 크롤러에서 수집한 리뷰 데이터 로드
# ============================================================

# JSON 라이브러리 임포트 (JSON 파일 읽기용)
import json

# 리뷰 데이터 로드
# - 파일 위치: ./res/reviews.json
# - 형식: [{"review": "...", "stars": 5, "date": "2025.09.01"}, ...]
with open('./res/reviews.json', 'r', encoding='utf-8') as f:
    review_list = json.load(f)

# ============================================================
# 데이터 확인: 총 개수 및 샘플 확인
# ============================================================
print(f"총 리뷰 개수: {len(review_list)}개\n")
print("=== 처음 3개 리뷰 샘플 ===")

# 처음 3개 리뷰 출력 (데이터 구조 확인용)
review_list[:3]

## 2. 데이터 로딩 및 탐색

`crawler.py`로 수집한 리뷰 데이터를 로드하고 구조를 확인합니다.

### 데이터 구조
```json
[
  {
    "review": "리뷰 텍스트",
    "stars": 5,
    "date": "2025.09.30"
  }
]
```

### 확인 항목
1. 총 리뷰 개수
2. 처음 3개 리뷰 샘플 (데이터 구조 검증)

In [ ]:
# ============================================================
# 데이터 검증: 별점 분포 및 Good/Bad 분류 확인
# ============================================================

from collections import Counter  # 별점 개수 세기용

# 별점별 개수 카운트 (0점: X개, 1점: X개, ...)
stars_count = Counter(r['stars'] for r in review_list)

# ============================================================
# 1. 별점 분포 출력
# ============================================================
print("=== 별점 분포 ===")
for stars in sorted(stars_count.keys()):
    count = stars_count[stars]
    percentage = (count / len(review_list)) * 100
    print(f"{stars}점: {count}개 ({percentage:.1f}%)")

# ============================================================
# 2. Good/Bad 분류 (4점 이상 = Good, 3점 이하 = Bad)
# ============================================================
# Good: 별점 4-5점 (긍정적 리뷰)
# Bad: 별점 0-3점 (부정적 리뷰 또는 크롤링 오류)
good_cnt = sum(1 for r in review_list if r['stars'] >= 4)
bad_cnt = sum(1 for r in review_list if r['stars'] < 4)

print(f"\n=== Good/Bad 분류 (4점 이상 기준) ===")
print(f"Good (4-5점): {good_cnt}개")
print(f"Bad (0-3점): {bad_cnt}개")

# ⚠️ 주의: 모두 0점이면 크롤러 별점 추출 실패
# ⚠️ 주의: 모두 5점이면 데이터 편향 (다양한 별점 필요)

In [ ]:
# === 데이터 품질 검증 (자동 경고 시스템) ===
from collections import Counter
import warnings

def validate_review_data(review_list):
    """
    리뷰 데이터의 품질을 검증하고 문제점을 경고
    
    검증 항목:
    1. 별점 분포 (편향 감지)
    2. Good/Bad 불균형
    3. 데이터 수집 품질
    """
    print("=" * 60)
    print("📊 데이터 품질 검증 리포트")
    print("=" * 60)
    
    # 1. 별점 분포 검증
    stars_dist = Counter(r['stars'] for r in review_list)
    unique_stars = len(stars_dist)
    
    print(f"\n✅ 1. 별점 분포")
    for stars in sorted(stars_dist.keys()):
        count = stars_dist[stars]
        pct = count / len(review_list) * 100
        print(f"   {stars}점: {count}개 ({pct:.1f}%)")
    
    # 경고: 별점이 1종류만 있음
    if unique_stars == 1:
        only_star = list(stars_dist.keys())[0]
        print(f"\n⚠️  [경고] 모든 리뷰가 {only_star}점입니다!")
        print(f"   → 데이터 편향 심각: 크롤러 URL 확인 필요")
        if only_star == 5:
            print(f"   → sort=HOST_CHOICE 사용 중일 가능성")
        elif only_star == 0:
            print(f"   → 별점 추출 실패: extract_stars() 함수 점검 필요")
    
    # 경고: 5점이 80% 이상
    if stars_dist.get(5, 0) / len(review_list) > 0.8:
        print(f"\n⚠️  [경고] 5점 리뷰가 {stars_dist[5]/len(review_list)*100:.1f}%입니다")
        print(f"   → 데이터 편향 가능성: 다양한 별점 확보 권장")
    
    # 2. Good/Bad 분류 검증
    good_cnt = sum(1 for r in review_list if r['stars'] >= 4)
    bad_cnt = sum(1 for r in review_list if r['stars'] < 4)
    
    print(f"\n✅ 2. Good/Bad 분류 (4점 이상 기준)")
    print(f"   Good: {good_cnt}개 ({good_cnt/len(review_list)*100:.1f}%)")
    print(f"   Bad: {bad_cnt}개 ({bad_cnt/len(review_list)*100:.1f}%)")
    
    # 경고: 한쪽이 0개
    if good_cnt == 0:
        print(f"\n❌ [오류] Good 리뷰가 없습니다!")
        print(f"   → 별점이 모두 3점 이하: 데이터 확인 필요")
    if bad_cnt == 0:
        print(f"\n❌ [오류] Bad 리뷰가 없습니다!")
        print(f"   → Good/Bad 비교 분석 불가능")
        print(f"   → 크롤러 URL 수정 권장: sort=LATEST 사용")
    
    # 경고: 심한 불균형 (90% 이상)
    if good_cnt > 0 and bad_cnt > 0:
        ratio = max(good_cnt, bad_cnt) / len(review_list)
        if ratio > 0.9:
            print(f"\n⚠️  [경고] 데이터 불균형 심각 ({ratio*100:.1f}%)")
            print(f"   → 소수 클래스 데이터 추가 수집 권장")
    
    # 3. 날짜 정보 검증
    dates_valid = [r['date'] for r in review_list if r['date']]
    print(f"\n✅ 3. 날짜 정보")
    print(f"   유효한 날짜: {len(dates_valid)}개")
    print(f"   누락된 날짜: {len(review_list) - len(dates_valid)}개")
    
    if dates_valid:
        from datetime import datetime
        try:
            # 가장 오래된 리뷰와 최신 리뷰
            dates_parsed = [datetime.strptime(d, '%Y.%m.%d') for d in dates_valid if '.' in d]
            if dates_parsed:
                oldest = min(dates_parsed).strftime('%Y.%m.%d')
                newest = max(dates_parsed).strftime('%Y.%m.%d')
                print(f"   수집 기간: {oldest} ~ {newest}")
        except:
            pass
    
    # 4. 종합 평가
    print(f"\n" + "=" * 60)
    issues = []
    if unique_stars == 1:
        issues.append("별점 편향")
    if bad_cnt == 0 or good_cnt == 0:
        issues.append("분류 불가능")
    if stars_dist.get(0, 0) > 0:
        issues.append("별점 추출 실패")
    
    if issues:
        print(f"❌ 데이터 품질: 문제 있음 ({', '.join(issues)})")
        print(f"   → 크롤러 재실행 권장")
    else:
        print(f"✅ 데이터 품질: 양호")
    
    print("=" * 60)
    
    return {
        'unique_stars': unique_stars,
        'good_count': good_cnt,
        'bad_count': bad_cnt,
        'has_issues': len(issues) > 0
    }

# 검증 실행
validation_result = validate_review_data(review_list)

In [ ]:
# === 초기 탐색: Good/Bad 리뷰 분리 (간단 버전) ===
# 이 셀은 초기 탐색용입니다. 실제 사용은 아래 preprocess_reviews() 함수를 사용하세요.

reviews_good, reviews_bad = [], []

for r in review_list:
    # stars == 5만 good으로 분류 (엄격한 기준)
    if r['stars'] == 5:
        reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
    else:
        reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

print(f"Good 리뷰: {len(reviews_good)}개")
print(f"Bad 리뷰: {len(reviews_bad)}개")
print("\n처음 3개 Bad 리뷰:")
reviews_bad[:3]

Good 리뷰: 0개
Bad 리뷰: 220개

처음 3개 Bad 리뷰:


['[REVIEW_START]저렴하게 잘 놀다 왔습니다.제방문 의자 있습니다[REVIEW_END]',
 '[REVIEW_START]호텔 내 주차장 공간이 좀 아쉬웠지만 만족했습니다.^^[REVIEW_END]',
 '[REVIEW_START]다 좋은데 객실에 친환경으로 슬리퍼 칫솔치약을 안주는데 슬리퍼만 줬으면 더 좋았을거같아요[REVIEW_END]']

In [ ]:
# === 데이터 검증: 날짜 및 리뷰 길이 통계 ===
import re

# 날짜 형식 분석
date_pattern = re.compile(r'\d{4}\.\d{2}\.\d{2}')
valid_dates = [r['date'] for r in review_list if date_pattern.match(r['date'])]
invalid_dates = [r['date'] for r in review_list if not date_pattern.match(r['date'])]

print("=== 날짜 형식 통계 ===")
print(f"유효한 날짜 (YYYY.MM.DD): {len(valid_dates)}개")
print(f"무효한 날짜: {len(invalid_dates)}개")
if invalid_dates[:5]:
    print(f"무효 날짜 예시: {invalid_dates[:5]}")

# 리뷰 길이 통계
review_lengths = [len(r['review']) for r in review_list]
avg_length = sum(review_lengths) / len(review_lengths) if review_lengths else 0
min_length = min(review_lengths) if review_lengths else 0
max_length = max(review_lengths) if review_lengths else 0

print(f"\n=== 리뷰 길이 통계 ===")
print(f"평균 길이: {avg_length:.1f}자")
print(f"최소 길이: {min_length}자")
print(f"최대 길이: {max_length}자")
print(f"30자 미만: {sum(1 for l in review_lengths if l < 30)}개")

=== 날짜 형식 통계 ===
유효한 날짜 (YYYY.MM.DD): 55개
무효한 날짜: 165개
무효 날짜 예시: ['루루*', '분홍색*******', '골져스 파셜오션 더블', '헤헤헤*******', '20시간 전']

=== 리뷰 길이 통계 ===
평균 길이: 82.7자
최소 길이: 10자
최대 길이: 933자
30자 미만: 85개


In [ ]:
# === 초기 탐색: 리뷰 텍스트 변환 ===
# 리스트를 줄바꿈으로 구분된 텍스트로 변환

reviews_good_text = '\n'.join(reviews_good)
reviews_bad_text = '\n'.join(reviews_bad)

# 처음 100자 확인 (전체 내용 미리보기)
print("=== Good 리뷰 텍스트 (처음 200자) ===")
print(reviews_good_text[:200] if reviews_good_text else "(비어있음)")
print(f"\n=== Bad 리뷰 텍스트 (처음 200자) ===")
print(reviews_bad_text[:200])

=== Good 리뷰 텍스트 (처음 200자) ===
(비어있음)

=== Bad 리뷰 텍스트 (처음 200자) ===
[REVIEW_START]저렴하게 잘 놀다 왔습니다.제방문 의자 있습니다[REVIEW_END]
[REVIEW_START]호텔 내 주차장 공간이 좀 아쉬웠지만 만족했습니다.^^[REVIEW_END]
[REVIEW_START]다 좋은데 객실에 친환경으로 슬리퍼 칫솔치약을 안주는데 슬리퍼만 줬으면 더 좋았을거같아요[REVIEW_END]
[REVIEW_START]


In [ ]:
# ============================================================
# 3-1. 전처리 함수: 초기 버전 (실험용)
# ============================================================

import datetime
from dateutil import parser

def preprocess_reviews_v1(path='./res/reviews.json'):
    """
    리뷰 JSON 파일을 전처리하여 good/bad 리뷰 **리스트**로 변환 (초기 버전)
    
    Args:
        path (str): 리뷰 JSON 파일 경로
    
    Returns:
        tuple: (reviews_good, reviews_bad) 
               - **리스트 형태** (각 요소는 '[REVIEW_START]...[REVIEW_END]' 문자열)
               - 최종 버전과 달리 텍스트로 변환하지 않음
    
    Note:
        ⚠️ 이 버전은 초기 실험용입니다. 
        실제 사용은 preprocess_reviews() 최종 버전을 사용하세요.
        
        주요 차이점:
        - v1: **리스트 반환** → summarize() 함수가 자동 변환
        - 최종: **텍스트 반환** → 바로 사용 가능
        
    전처리 과정:
        1. JSON 파일에서 리뷰 데이터 로드
        2. 각 리뷰의 날짜 파싱 (실패 시 현재 날짜로 간주)
        3. 별점 기준으로 분류:
           - stars >= 4: good 리뷰
           - stars 1-3: bad 리뷰
           - stars == 0: bad 리뷰 (크롤러 오류로 간주)
        4. '[REVIEW_START]...[REVIEW_END]' 포맷으로 래핑
    """
    # JSON 파일 로드
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)

    reviews_good, reviews_bad = [], []

    # 현재 날짜와 6개월 전 날짜 계산 (날짜 필터용, 현재는 미사용)
    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)

    for r in review_list:
        review_date_str = r['date']
        
        # 날짜 파싱 시도
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            # 파싱 실패 시 현재 날짜로 간주 (최근 리뷰로 처리)
            review_date = current_date

        # 날짜 필터 (주석 처리됨 - 모든 리뷰 포함)
        # if review_date < date_boundary:
        #     continue

        # 별점 기준으로 분류
        # stars >= 4: good, stars 1-3: bad, stars == 0: bad (크롤러 오류)
        if r['stars'] >= 4:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        elif r['stars'] > 0:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            # stars == 0인 경우 크롤러 오류로 간주하고 bad로 분류
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

    return reviews_good, reviews_bad  # 리스트로 반환 ⬅️

# ============================================================
# 테스트 실행
# ============================================================
good, bad = preprocess_reviews_v1()
print(f"좋은 리뷰: {len(good)}개 (리스트 형태)")
print(f"나쁜 리뷰: {len(bad)}개 (리스트 형태)")
print("\n=== 처음 3개 리뷰 샘플 ===")
for i, review in enumerate(bad[:3], 1):
    print(f"\n[{i}] {review}")

## 3. 전처리 함수 정의

리뷰 데이터를 요약에 적합한 형태로 전처리하는 함수들입니다.

### 버전 비교

| 항목 | v1 (초기 버전) | v2 (최종 버전) |
|------|---------------|---------------|
| **반환 형태** | 리스트 (`list`) | 텍스트 (`str`) |
| **날짜 필터** | ❌ 없음 | ✅ 6개월 이내 |
| **길이 필터** | ❌ 없음 | ✅ 30자 이상 |
| **개수 제한** | ❌ 없음 | ✅ 최대 50개 |
| **Good 기준** | `stars >= 4` | `stars == 5` (엄격) |
| **용도** | 초기 실험 | 프로덕션 |

---

### 3-1. 초기 버전 (실험용)

기본적인 전처리 기능만 제공하는 초기 버전입니다.

In [ ]:
# ============================================================
# 평가 함수: GPT-4를 사용한 요약 품질 비교
# ============================================================

def pairwise_eval(reviews, answer_a, answer_b):
    """
    두 개의 요약 결과를 GPT-4로 비교 평가하는 함수
    
    목적:
        사람 대신 GPT-4를 심사위원으로 활용하여 두 요약의 품질을 비교
        (사람이 평가하려면 시간과 비용이 많이 듦)
    
    Args:
        reviews (str): 원본 리뷰 텍스트 (평가 기준)
        answer_a (str): 평가할 요약 A (예: Baseline 방법)
        answer_b (str): 평가할 요약 B (예: 개선된 방법)
    
    Returns:
        Completion: OpenAI API 응답 객체
        
    Note:
        평가 결과 형식:
        - [[A]]: A가 더 좋음
        - [[B]]: B가 더 좋음
        - [[C]]: 무승부 (비슷함)
        
    평가 기준:
        - 유용성 (helpfulness)
        - 관련성 (relevance)
        - 정확성 (accuracy)
        - 깊이 (depth)
        - 창의성 (creativity)
        - 세부사항 수준 (level of detail)
    """
    # ============================================================
    # GPT-4 평가 프롬프트 (공정한 심사 요청)
    # ============================================================
    eval_prompt = f"""[System]
Please act as an impartial judge and evaluate the quality of the Korean summaries provided by two
AI assistants to the set of user reviews on accommodations displayed below. You should choose the assistant that
follows the user's instructions and answers the user's question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.

[User Reviews]
{reviews}

[The Start of Assistant A's Answer]
{answer_a}
[The End of Assistant A's Answer]

[The Start of Assistant B's Answer]
{answer_b}
[The End of Assistant B's Answer]"""
    
    # ============================================================
    # GPT-4o 모델로 평가 수행
    # ============================================================
    # GPT-4o 사용 이유: GPT-3.5보다 평가 능력이 우수하고 객관적
    completion = client.chat.completions.create(
        model='gpt-4o-2024-05-13',  # GPT-4o 모델
        messages=[{'role': 'user', 'content': eval_prompt}],
        temperature=0.0  # 일관성을 위해 temperature=0 (항상 같은 평가)
    )

    return completion

## 4. 요약 및 평가 함수

OpenAI API를 사용하여 리뷰를 요약하고, 두 요약을 비교 평가하는 함수들입니다.

### 주요 함수

#### 1. `summarize(reviews, prompt, temperature, model)`
- **목적**: 리뷰 텍스트를 요약 생성
- **모델**: `gpt-3.5-turbo-0125` (빠르고 저렴) 또는 `gpt-4o-2024-05-13` (고품질)
- **temperature**: 0.0 (결정적) ~ 1.0 (창의적)

#### 2. `pairwise_eval(reviews, answer_a, answer_b)`
- **목적**: 두 요약을 GPT-4로 비교 평가
- **심사위원**: GPT-4o (사람 대신 공정한 평가)
- **결과**: `[[A]]` (A가 더 좋음), `[[B]]` (B가 더 좋음), `[[C]]` (무승부)

#### 3. `pairwise_eval_batch(reviews, answers_a, answers_b)`
- **목적**: 여러 요약 쌍을 배치로 평가
- **용도**: 통계적으로 유의미한 비교 (10개 이상 권장)

In [ ]:
# ============================================================
# 4. 요약 함수 정의
# ============================================================

# 리뷰 데이터 로드 (초기 버전 사용 - Good 리뷰만)
reviews, _ = preprocess_reviews_v1(path='./res/reviews.json')

def summarize(reviews, prompt, temperature=0.0, model='gpt-3.5-turbo-0125'):
    """
    OpenAI API를 사용하여 리뷰 텍스트를 요약하는 함수
    
    Args:
        reviews (str or list): 요약할 리뷰 텍스트 또는 리뷰 리스트
            - 문자열: 줄바꿈으로 구분된 리뷰 텍스트
            - 리스트: 리뷰 문자열 리스트 (자동으로 텍스트로 변환)
        prompt (str): 요약 생성을 위한 프롬프트 (지시사항)
        temperature (float): 생성 다양성 조절 (0.0-1.0)
            - 0.0: 항상 같은 답변 (결정적)
            - 1.0: 다양한 답변 (창의적)
        model (str): 사용할 OpenAI 모델 이름
            - 'gpt-3.5-turbo-0125': 빠르고 저렴함
            - 'gpt-4o-2024-05-13': 더 정확하지만 비쌈
    
    Returns:
        Completion: OpenAI API 응답 객체
            - .choices[0].message.content로 요약 텍스트 추출
    """
    # ============================================================
    # 1. 리뷰 데이터 전처리
    # ============================================================
    # 리뷰가 리스트인 경우 → 줄바꿈으로 구분된 텍스트로 변환
    if isinstance(reviews, list):
        reviews = '\n'.join(reviews)
    
    # ============================================================
    # 2. 프롬프트 생성
    # ============================================================
    # 프롬프트 + 리뷰 텍스트 결합
    full_prompt = prompt + '\n\n' + reviews

    # ============================================================
    # 3. OpenAI API 호출
    # ============================================================
    completion = client.chat.completions.create(
        model=model,  # 모델 선택
        messages=[{'role': 'user', 'content': full_prompt}],  # 메시지 형식
        temperature=temperature  # 다양성 조절
    )

    return completion

# ============================================================
# Baseline 프롬프트 정의 (가장 단순한 형태)
# ============================================================
PROMPT_BASELINE = """아래 숙소 리뷰에 대해 5문장 내로 요약해줘:"""

# ============================================================
# 테스트 실행: Baseline 프롬프트로 요약 생성
# ============================================================
print("=== Baseline 요약 테스트 ===")
print(summarize(reviews, PROMPT_BASELINE).choices[0].message.content)

## 5. 실험: Baseline

가장 기본적인 프롬프트로 리뷰 요약을 시도합니다.

### Baseline 프롬프트
```
아래 숙소 리뷰에 대해 5문장 내로 요약해줘:
```

### 실험 설정
- **모델**: GPT-3.5 Turbo
- **temperature**: 1.0 (다양한 요약 생성)
- **생성 개수**: 10개
- **목적**: 기본 성능 측정 (비교 기준선)

In [ ]:
# Baseline 요약 생성 (10개, temperature=1.0으로 다양성 확보)
eval_count = 10

summaries_baseline = [
    summarize(reviews, PROMPT_BASELINE, temperature=1.0).choices[0].message.content 
    for _ in range(eval_count)
]

print(f"=== Baseline 요약 {eval_count}개 생성 완료 ===")
print(f"\n첫 번째 요약:\n{summaries_baseline[0]}")
summaries_baseline

=== Baseline 요약 10개 생성 완료 ===

첫 번째 요약:
이 숙소는 위치가 좋고 깔끔하며 직원들이 친절했다. 그러나 시설은 조금 오래되었고 소음이 조금 심했다. 또한 조식은 다양하지 않았지만 맛은 괜찮았다. 전체적으로 만족스러운 숙박 경험이었다.


['이 숙소는 위치가 좋고 깔끔하며 직원들이 친절했다. 그러나 시설은 조금 오래되었고 소음이 조금 심했다. 또한 조식은 다양하지 않았지만 맛은 괜찮았다. 전체적으로 만족스러운 숙박 경험이었다.',
 '좋은 위치와 깔끔한 시설을 갖춘 이 숙소는 편안하고 안락한 숙박을 경험할 수 있는 곳이다. 친절한 직원들이 손님을 맞아주며 청결한 객실과 편안한 침대가 눈에 띈다. 조식 뷔페는 다양하고 맛있었고, 숙소 주변에는 맛집과 편의시설이 가까이 있어 편리했다. 다음에 또 방문하고 싶은 만족스러운 숙소였다. 강력 추천한다.',
 '이 숙소는 매우 깨끗하고 편안한 분위기를 가지고 있으며, 위치도 편리하고 주변에 맛집이 많아 좋았다. 직원들의 친절함과 서비스도 훌륭했고, 객실 내 모든 시설이 잘 구비되어 있어 편리했다. 다만 조식 메뉴가 다양하지는 않았지만 맛은 괜찮았다. 전체적으로 만족스러운 숙박이었고 다음에 또 방문하고 싶다.',
 '좋은 위치와 깔끔한 시설을 갖춘 이 숙소는 친절한 직원들이 있어 편안한 숙박을 제공합니다. 객실은 넓고 아늑하며 모든 시설이 잘 갖춰져 있습니다. 조식 뷔페가 매우 훌륭하고 다양한 메뉴를 즐길 수 있습니다. 호텔 주변에는 편의시설이 많아 편리한 여행을 즐길 수 있습니다. 전체적으로 만족스러운 숙박이었고 다시 방문하고 싶은 곳입니다.',
 '이 숙소는 깨끗하고 아늑한 분위기였지만 침대가 조금 불편했고 소음이 조금 심했다.조식은 맛있었으나 다양성이 부족했으며 직원들의 서비스는 친절하지만 조금 느린 편이었다. 위치는 관광지와 가깝지만 도보로 이동하기에는 조금 먼 편이었다. 전체적으로 만족스러운 경험이었지만 몇 가지 부분에서 개선이 필요할 것 같다.',
 '이 숙소는 깨끗하고 아늑하여 편안한 숙박을 즐길 수 있는 곳이었어요. 주인이 친절하고 세심한 배려가 느껴졌고, 아침식사도 맛있었습니다. 숙소 위치가 번화가와 가까워 이동이 편리했고, 주변에 맛집과 편의시설이 많았습니다. 다음에 또 이용하고 싶은 곳이에요. 강력 추천합니다!',
 '쾌적한 분위기와 청

In [ ]:
# ============================================================
# 배치 평가 함수: 여러 요약 쌍을 한 번에 평가
# ============================================================

from tqdm import tqdm

def pairwise_eval_batch(reviews, answers_a, answers_b):
    """
    여러 요약 쌍을 배치로 평가하는 함수 (GPT-4 심사위원)
    
    목적:
        실험별로 10개 이상의 요약을 생성하여 통계적으로 유의미한 비교
        (단일 평가는 우연에 좌우될 수 있음)
    
    Args:
        reviews (str): 원본 리뷰 텍스트 (모든 평가에 동일하게 사용)
        answers_a (list): 평가할 요약 A 리스트 (예: Baseline 방법 10개)
        answers_b (list): 평가할 요약 B 리스트 (예: 개선 방법 10개)
    
    Returns:
        tuple: (a_cnt, b_cnt, draw_cnt)
            - a_cnt: A가 더 좋았던 횟수
            - b_cnt: B가 더 좋았던 횟수
            - draw_cnt: 무승부 횟수
    
    Note:
        평가 프로세스:
        1. 각 쌍(A[i], B[i])을 pairwise_eval()로 평가
        2. GPT-4가 [[A]], [[B]], [[C]] 중 하나 반환
        3. 결과를 카운트하여 통계 생성
        
        실행 시간:
        - 10쌍 평가 시 약 30-60초 소요
        - API 비용 발생 (GPT-4o 사용)
    """
    a_cnt, b_cnt, draw_cnt = 0, 0, 0
    
    # 진행 상황 표시와 함께 평가 (tqdm 프로그레스바)
    for i in tqdm(range(len(answers_a)), desc="평가 진행"):
        # 두 요약을 비교 평가 (GPT-4o가 심사)
        completion = pairwise_eval(reviews, answers_a[i], answers_b[i])
        verdict_text = completion.choices[0].message.content

        # ============================================================
        # 평가 결과 파싱 (GPT-4의 최종 판결 추출)
        # ============================================================
        if '[[A]]' in verdict_text:
            a_cnt += 1  # A가 더 좋음
        elif '[[B]]' in verdict_text:
            b_cnt += 1  # B가 더 좋음
        elif '[[C]]' in verdict_text:
            draw_cnt += 1  # 무승부 (비슷함)
        else:
            # 예상치 못한 형식 (드문 경우)
            print(f'\n[경고] 평가 {i}: 결과 파싱 실패')
            print(f'응답 내용: {verdict_text[:100]}...')

    return a_cnt, b_cnt, draw_cnt

# ============================================================
# 사용 예시 (주석 처리됨)
# ============================================================
# 주의: summary_real_20240526이 정의되어 있어야 실행 가능
# (실제 프로덕션 요약과 비교하기 위한 변수)
# 
# wins, losses, ties = pairwise_eval_batch(
#     reviews,  # 원본 리뷰
#     summaries_baseline,  # Baseline 요약 10개
#     [summary_real_20240526 for _ in range(len(summaries_baseline))]  # 실제 요약 10개 (동일)
# )
# print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')
# 
# 결과 해석:
# - Wins: Baseline이 실제 요약보다 좋았던 횟수
# - Losses: 실제 요약이 Baseline보다 좋았던 횟수
# - Ties: 비슷했던 횟수

In [ ]:
# ============================================================
# Prompt Engineering 실험: 구체적인 조건 추가
# ============================================================

# 개선된 프롬프트 (구체적인 조건 추가)
prompt = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

# ============================================================
# 10개 요약 생성 (temperature=1.0으로 다양성 확보)
# ============================================================
eval_count = 10
summaries = [
    summarize(reviews, prompt, temperature=1.0).choices[0].message.content 
    for _ in range(eval_count)
]

print(f"=== Prompt Engineering 요약 {eval_count}개 생성 완료 ===")
print(f"\n첫 번째 요약:\n{summaries[0]}")

# ============================================================
# 평가 (주석 처리됨)
# ============================================================
# 주의: summary_real_20240526이 정의되어 있어야 실행 가능
# (실제 프로덕션 요약과 비교하기 위한 변수)
# 
# wins, losses, ties = pairwise_eval_batch(
#     reviews,  # 원본 리뷰
#     summaries,  # Prompt Engineering 요약 10개
#     [summary_real_20240526 for _ in range(len(summaries))]  # 실제 요약 10개
# )
# print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')
# 
# 예상 결과:
# - Baseline보다 향상: 톤앤매너, 문장 길이 제한 효과
# - 실제 요약과 비교: 품질 개선 확인

## 6. 실험: Prompt Engineering

프롬프트에 **구체적인 조건**을 추가하여 요약 품질을 개선합니다.

### 개선된 프롬프트 조건
1. ✅ **존댓말 사용**: 모든 문장은 존댓말로 끝남
2. ✅ **톤앤매너**: 숙소 소개 형식
   - ✅ 좋은 예시: "좋은 숙소였다는 평입니다"
   - ❌ 나쁜 예시: "좋은 숙소였습니다"
3. ✅ **문장 길이**: 최소 2문장, 최대 5문장

### 기대 효과
- Baseline 대비 **톤앤매너 일관성** 향상
- **적절한 문장 길이** 유지
- **전문적인 소개 형식** 적용

### 실험 설정
- **생성 개수**: 10개
- **API 비용**: 약 30-60초 소요

In [ ]:
# ============================================================
# 3-2. 전처리 함수: 최종 버전 (프로덕션용)
# ============================================================

import datetime
from dateutil import parser

def preprocess_reviews(path='./res/reviews.json'):
    """
    리뷰 JSON 파일을 전처리하여 good/bad 리뷰 **텍스트**로 변환 (최종 버전)
    
    Args:
        path (str): 리뷰 JSON 파일 경로
    
    Returns:
        tuple: (reviews_good_text, reviews_bad_text) 
               - **텍스트 형태** (줄바꿈으로 구분된 단일 문자열)
               - summarize() 함수에 바로 전달 가능
    
    Note:
        이 함수는 프로덕션용 최종 버전입니다.
        
        적용된 필터:
        - ✅ 6개월 이내 리뷰만 사용 (최신성 확보)
        - ✅ 30자 미만 리뷰 제외 (품질 확보)
        - ✅ 최대 50개로 제한 (OpenAI API 토큰 절약)
        - ✅ stars == 5만 good (엄격한 기준)
        
        v1 버전과 차이:
        - v1: 리스트 반환, 필터 없음
        - 최종: **텍스트 반환**, 다양한 필터 적용
        
    전처리 과정:
        1. JSON 파일에서 리뷰 데이터 로드
        2. 날짜 파싱 및 6개월 이내 필터 적용
        3. 30자 미만 짧은 리뷰 필터링
        4. 별점 기준으로 분류 (stars == 5만 good)
        5. 최대 50개로 제한 (토큰 절약)
        6. 텍스트 형태로 변환 (줄바꿈 구분)
    """
    # JSON 파일 로드
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)

    reviews_good, reviews_bad = [], []

    # 현재 날짜와 6개월 전 날짜 계산 (최근 리뷰 우선)
    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)  # 6개월 = 180일

    filtered_cnt = 0  # 필터링된 리뷰 개수
    
    for r in review_list:
        review_date_str = r['date']
        
        # 날짜 파싱 시도
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            # 파싱 실패 시 현재 날짜로 간주
            review_date = current_date

        # 필터 1: 6개월 이전 리뷰 제외
        if review_date < date_boundary:
            continue
        
        # 필터 2: 30자 미만 리뷰 제외 (의미 있는 리뷰만 선택)
        if len(r['review']) < 30:
            filtered_cnt += 1
            continue

        # 별점 기준으로 분류
        # stars == 5만 good (매우 만족한 리뷰만)
        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            # stars 1-4 또는 0 (크롤러 오류)
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

    # 필터 3: 최대 50개로 제한 (OpenAI API 토큰 제한 대응)
    reviews_good = reviews_good[:
                                min(len(reviews_good), 50)]
    reviews_bad = reviews_bad[:min(len(reviews_bad), 50)]

    # 리스트를 텍스트로 변환 (줄바꿈 구분) ⬅️ 최종 버전의 핵심
    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)

    # ============================================================
    # 전처리 결과 출력
    # ============================================================
    print(f"=== 전처리 결과 ===")
    print(f"Good 리뷰: {len(reviews_good)}개 (텍스트 형태)")
    print(f"Bad 리뷰: {len(reviews_bad)}개 (텍스트 형태)")
    print(f"필터링됨 (30자 미만): {filtered_cnt}개")

    return reviews_good_text, reviews_bad_text  # 텍스트로 반환 ⬅️

# ============================================================
# 최종 버전으로 리뷰 로드 (요약 작업에 사용)
# ============================================================
reviews, _ = preprocess_reviews()

### 3-2. 최종 버전 (프로덕션용)

날짜 필터, 리뷰 길이 필터, 최대 개수 제한 등이 추가된 **프로덕션용 최종 버전**입니다.

#### 적용된 필터
1. ✅ **6개월 이내 리뷰만 사용** (최신성 확보)
2. ✅ **30자 미만 짧은 리뷰 제외** (품질 확보)
3. ✅ **최대 50개로 제한** (OpenAI API 토큰 절약)
4. ✅ **stars == 5만 Good** (엄격한 기준)

#### 주요 차이점
- **반환 형태**: 리스트 → **텍스트** (줄바꿈 구분)
- **summarize() 함수에 바로 전달 가능**

In [ ]:
# ===== 평가 실험 (주석 처리됨) =====
# 이 셀은 실제 프로덕션 요약과 비교하기 위한 평가 코드입니다.
# summary_real_20240526 변수(실제 프로덕션 요약)가 필요합니다.
# 
# 실행 방법:
# 1. summary_real_20240526 변수를 정의하세요
# 2. 아래 코드의 주석을 제거하고 실행하세요
#
# eval_count = 10
# summaries = [summarize(reviews, prompt, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
# wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
# print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

print("이 셀은 주석 처리되어 있습니다. summary_real_20240526을 정의 후 실행하세요.")

이 셀은 주석 처리되어 있습니다. summary_real_20240526을 정의 후 실행하세요.


In [ ]:
# === 1-shot 예시 데이터 준비 ===
# 다른 숙소의 리뷰로 고품질 요약 예시를 생성합니다.

try:
    # 1-shot 예시 데이터 로드
    reviews_1shot, _ = preprocess_reviews('./res/ninetree_pangyo.json')

    # GPT-4로 고품질 요약 생성 (few-shot 예시용)
    summary_1shot = summarize(
        reviews_1shot, 
        prompt, 
        temperature=0.0, 
        model='gpt-4-turbo-2024-04-09'
    ).choices[0].message.content

    print(f"=== 1-shot 예시 요약 ===\n{summary_1shot}\n")

    # 1-shot 프롬프트 생성
    prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.

다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

    # 10개 요약 생성
    summaries = [
        summarize(reviews, prompt_1shot, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content 
        for _ in range(eval_count)
    ]

    print(f"=== 1-shot Learning 요약 {eval_count}개 생성 완료 ===")
    print(f"\n첫 번째 요약:\n{summaries[0]}")

except FileNotFoundError:
    print("⚠️ ./res/ninetree_pangyo.json 파일이 없습니다.")
    print("1-shot 학습을 건너뜁니다. (선택 사항)")
    print("\n파일을 준비하려면:")
    print("1. 야놀자 나인트리 판교 숙소 리뷰를 크롤링")
    print("2. ./res/ninetree_pangyo.json으로 저장")

# 주의: summary_real_20240526이 정의되어 있어야 실행 가능
# wins, losses, ties = pairwise_eval_batch(
#     reviews, 
#     summaries, 
#     [summary_real_20240526 for _ in range(len(summaries))]
# )
# print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

⚠️ ./res/ninetree_pangyo.json 파일이 없습니다.
1-shot 학습을 건너뜁니다. (선택 사항)

파일을 준비하려면:
1. 야놀자 나인트리 판교 숙소 리뷰를 크롤링
2. ./res/ninetree_pangyo.json으로 저장


## 7. 실험: 1-shot Learning

다른 숙소의 리뷰 요약 **예시 1개**를 제공하여 모델 성능을 개선합니다.

### Few-shot Learning이란?
- **0-shot**: 예시 없이 바로 작업
- **1-shot**: 예시 1개 제공
- **2-shot**: 예시 2개 제공
- **효과**: 예시를 통해 모델이 원하는 형식을 더 잘 이해함

### 1-shot 예시 준비
1. 다른 숙소 (나인트리 판교) 리뷰 로드
2. GPT-4로 **고품질 요약** 생성
3. 예시와 함께 프롬프트 제공

### 필요 파일 (선택 사항)
- `./res/ninetree_pangyo.json` (1-shot 예시용)
- ⚠️ 파일이 없어도 실험은 건너뜁니다

In [ ]:
# === 2-shot 예시 데이터 준비 ===
# 두 번째 숙소의 리뷰로 추가 예시를 생성합니다.

try:
    # 2-shot 예시 데이터 준비 (두 번째 숙소)
    reviews_2shot, _ = preprocess_reviews('./res/ninetree_yongsan.json')

    # GPT-4로 두 번째 고품질 요약 생성
    summary_2shot = summarize(
        reviews_2shot, 
        prompt_1shot,  # 1-shot 프롬프트 사용
        temperature=0.0, 
        model='gpt-4-turbo-2024-04-09'
    ).choices[0].message.content

    print(f"=== 2-shot 두 번째 예시 요약 ===\n{summary_2shot}\n")

    # 2-shot 프롬프트 생성 (두 개의 예시 포함)
    prompt_2shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 리뷰들과 요약 예시입니다.

예시 리뷰들 1:
{reviews_1shot}
예시 요약 결과 1:
{summary_1shot}

예시 리뷰들 2:
{reviews_2shot}
예시 요약 결과 2:
{summary_2shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

    # 10개 요약 생성
    summaries = [
        summarize(reviews, prompt_2shot, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content 
        for _ in range(eval_count)
    ]

    print(f"=== 2-shot Learning 요약 {eval_count}개 생성 완료 ===")
    print(f"\n첫 번째 요약:\n{summaries[0]}")

except FileNotFoundError:
    print("⚠️ ./res/ninetree_yongsan.json 파일이 없습니다.")
    print("2-shot 학습을 건너뜁니다. (선택 사항)")
    print("\n파일을 준비하려면:")
    print("1. 야놀자 나인트리 용산 숙소 리뷰를 크롤링")
    print("2. ./res/ninetree_yongsan.json으로 저장")
except NameError as e:
    print(f"⚠️ 변수 오류: {e}")
    print("1-shot 학습 셀을 먼저 실행해주세요. (reviews_1shot, summary_1shot 필요)")

# 주의: summary_real_20240526이 정의되어 있어야 실행 가능
# wins, losses, ties = pairwise_eval_batch(
#     reviews, 
#     summaries, 
#     [summary_real_20240526 for _ in range(len(summaries))]
# )
# print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

⚠️ ./res/ninetree_yongsan.json 파일이 없습니다.
2-shot 학습을 건너뜁니다. (선택 사항)

파일을 준비하려면:
1. 야놀자 나인트리 용산 숙소 리뷰를 크롤링
2. ./res/ninetree_yongsan.json으로 저장


## 8. 실험: 2-shot Learning

다른 숙소의 리뷰 요약 **예시 2개**를 제공하여 모델 성능을 더욱 개선합니다.

### 2-shot Learning의 장점
- 1-shot보다 **더 많은 예시** 제공
- 모델이 **패턴을 더 잘 학습**
- **일관성 있는 요약** 생성 가능

### 2-shot 예시 준비
1. 첫 번째 숙소 (나인트리 판교) 예시 사용
2. 두 번째 숙소 (나인트리 용산) 리뷰 로드
3. GPT-4로 **두 번째 고품질 요약** 생성
4. **두 개의 예시**와 함께 프롬프트 제공

### 필요 파일 (선택 사항)
- `./res/ninetree_pangyo.json` (1번째 예시)
- `./res/ninetree_yongsan.json` (2번째 예시)
- ⚠️ 파일이 없어도 실험은 건너뜁니다

---

## 🎉 실험 완료

Baseline → Prompt Engineering → 1-shot → 2-shot 순서로  
프롬프트 최적화 과정을 학습했습니다!

In [ ]:
summaries

['1. 숙소는 깨끗하고 조용해서 편히 쉴 수 있었습니다.\n2. 위치가 편리하고 주변에 훌륭한 레스토랑도 많아서 만족스러웠습니다.\n3. 직원들이 친절하고 서비스도 좋아서 다음에 또 방문하고 싶은 숙소였습니다.',
 '리뷰 1: 전체적으로 깔끔하고 편안한 숙소였습니다. 직원들의 친절한 서비스도 기억에 남아요.\n\n리뷰 2: 위치가 편리하고 주변에 맛집이 많아 좋았습니다. 다만 룸 컨디션이 조금 떨어지는 부분이 있었어요.\n\n요약: 깔끔하고 편안한 숙소로서, 주변 맛집이 풍부한 위치가 장점입니다. 직원들의 친절한 서비스도 좋았으나 룸 컨디션에 조금 개선이 필요한 부분이 있습니다. 재방문을 고려해 볼 만한 숙박 시설입니다.',
 '1. 시설이 깔끔하고 직원들이 친절해서 편안한 숙박이 되었습니다.\n2. 조식도 맛있고 전반적으로 만족스러웠습니다.\n3. 위치가 번화가와 가까워 편리했으며 다음에 또 방문하고 싶은 숙소입니다.',
 '리뷰: "이번 숙박은 정말 편안했어요. 직원들이 매우 친절하고 시설도 깨끗하고 편리했습니다. 다음에 또 방문하고 싶네요."\n리뷰: "위치도 좋고 가격대비 만족스러운 숙소였어요. 조식도 맛있었고 편안한 휴식을 취할 수 있었습니다."\n리뷰: "이번 숙소는 전반적으로 좋았지만 욕조가 조금 작았던 점이 아쉬웠어요. 그래도 다음에 기회가 된다면 재방문을 고려하겠습니다."\n\n요약: \n전반적으로 편안하고 친절한 직원들이 있는 깨끗하고 편리한 숙소였습니다. 다음에 또 방문하고 싶은 만족스러운 숙소입니다. 욕조가 조금 작았지만 위치도 좋고 조식도 맛있어서 편안한 휴식을 취할 수 있는 숙소입니다.재방문을 고려해볼만한 좋은 숙박 경험이었습니다.',
 '숙소는 전반적으로 청결하고 편안한 분위기를 가졌습니다. 직원들의 친절한 서비스도 인상적이었고 편안한 휴식을 취할 수 있었습니다. 또한, 가격 대비 만족도가 높아 재방문하고 싶은 마음이 들었습니다. 전체적으로 좋은 경험을 했고, 추천할 만한 숙소입니다.',
 '1. 객실이 깨끗하고 편안했으며 직원들